# Transfer Learning con EfficientNet



In [1]:
!pip -q install tensorflow_datasets


In [2]:
# ============================================================
# LIBRERIAS
# ============================================================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import itertools, time
import pandas as pd
import seaborn as sns
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [3]:

# =============================================================================
# ESTILOS VISUALES PARA LA TERMINAL (CLI)
# =============================================================================

class Colors:
    """
    Clase para guardar los códigos de escape ANSI para dar color a la salida en la terminal.
    """
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'      # Código para resetear el color
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def print_header(title):
    """Imprime un encabezado principal estilizado."""
    print(f"\n{Colors.BOLD}{Colors.HEADER}{'='*60}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.HEADER}📊 {title.upper()}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.HEADER}{'='*60}{Colors.ENDC}")

def print_subheader(title):
    """Imprime un subencabezado estilizado."""
    print(f"\n{Colors.BOLD}{Colors.OKCYAN}🔹 {title}{Colors.ENDC}")
    print(f"{Colors.OKCYAN}{'-'*40}{Colors.ENDC}")

def print_info(message):
    """Imprime un mensaje informativo."""
    print(f"{Colors.OKBLUE}ℹ️  {message}{Colors.ENDC}")

def print_success(message):
    """Imprime un mensaje de éxito."""
    print(f"{Colors.OKGREEN}✅ {message}{Colors.ENDC}")

def print_warning(message):
    """Imprime un mensaje de advertencia."""
    print(f"{Colors.WARNING}⚠️  {message}{Colors.ENDC}")

def print_error(message):
    """Imprime un mensaje de error."""
    print(f"{Colors.FAIL}❌ {message}{Colors.ENDC}")

def print_text(message):
    """Imprime un texto normal, sin color ni íconos."""
    print(message)

# =============================================================================
# ESTILOS PARA GRÁFICAS (MATPLOTLIB / SEABORN)
# =============================================================================

def get_plot_colors(n_colors=4):
    """
    Genera una paleta de colores profesional para usar en gráficas.

    Args:
        n_colors (int): El número de colores que necesitas.

    Returns:
        list: Una lista de códigos de color hexadecimales.
    """
    # Puedes cambiar "flare" por otras paletas de seaborn como "crest", "mako", "viridis", etc.
    return sns.color_palette("flare", n_colors).as_hex()



In [4]:
# ============================================================
# Configuración reproducible y parámetros
# ============================================================
SEED = 42
tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Acelerar con XLA (compilación del grafo)
try:
    tf.config.optimizer.set_jit(True)
    print("✅ XLA/JIT activado")
except Exception as e:
    print("⚠️ No se pudo activar XLA/JIT:", e)

# Variables de control
USE_EFFICIENTNET = True   # Para usar ResNet50 cambiamos a False
IMG_SIZE = (224, 224)
NUM_CLASSES = 10
RANDOM_STATE = 42
BATCH_SIZE = 64
EPOCHS_FREEZE = 5         # Fase 1: base congelada
EPOCHS_FINETUNE = 3       # Fase 2: fine-tuning
VAL_SPLIT = 0.2



if USE_EFFICIENTNET:
    from tensorflow.keras.applications.efficientnet import EfficientNetB0 as BASE, preprocess_input

print(tf.__version__)


✅ XLA/JIT activado
2.19.0


## Carga de datos
Por defecto se usa **CIFAR-10** para simplificar la ejecución sin rutas locales.

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.flatten(); y_test = y_test.flatten()
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

n_train, n_test = x_train.shape[0], x_test.shape[0]
num_classes_detected = np.unique(y_train).size
assert num_classes_detected == NUM_CLASSES, f"NUM_CLASSES={NUM_CLASSES} pero se detectaron {num_classes_detected}"

#Tabla resumen de dimensiones globales
df_summary = pd.DataFrame({
    "Conjunto": ["Entrenamiento", "Prueba"],
    "x_shape": [x_train.shape, x_test.shape],
    "y_shape": [y_train.shape, y_test.shape],
    "Cantidad": [n_train, n_test]
})

#Distribución de clases (entrenamiento)
unique, counts = np.unique(y_train, return_counts=True)
df_dist = pd.DataFrame({"Clase": unique.flatten(), "Cantidad": counts})
df_dist["Proporción"] = (df_dist["Cantidad"] / n_train).round(4)


# División train/val
num_train = int((1 - VAL_SPLIT) * len(x_train))
x_tr, y_tr = x_train[:num_train], y_train[:num_train]
x_val, y_val = x_train[num_train:], y_train[num_train:]
len(x_tr), len(x_val), len(x_test)

print_header(f"Train: {len(x_tr)} | Val: {len(x_val)} | Test: {len(x_test)}")

#Crear DataFrame de muestra
sample_size = 5
df_sample = pd.DataFrame({
    "Dimensión": [x_tr[:sample_size], x_val[:sample_size], x_test[:sample_size]],
    "Etiqueta": [y_tr[:sample_size], y_val[:sample_size], y_test[:sample_size]]})
df_sample["Etiqueta"] = df_sample["Etiqueta"].apply(lambda x: [class_names[i] for i in x])
df_sample["Dimensión"] = df_sample["Dimensión"].apply(lambda x: [f"{i.shape[0]}x{i.shape[1]}" for i in x])



print_subheader("Resumen de Dimensiones Globales")
display(df_summary)

print_subheader("Distribución de Clases (Entrenamiento)")
display(df_dist)

print_subheader("Muestra de imágenes")
print_info(df_sample)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step

📊 TRAIN: 40000 | VAL: 10000 | TEST: 10000

🔹 Resumen de Dimensiones Globales
----------------------------------------


,Conjunto,x_shape,y_shape,Cantidad
0,Entrenamiento,"(50000, 32, 32, 3)","(50000,)",50000
1,Prueba,"(10000, 32, 32, 3)","(10000,)",10000



🔹 Distribución de Clases (Entrenamiento)
----------------------------------------


,Clase,Cantidad,Proporción
0,0,5000,0.1
1,1,5000,0.1
2,2,5000,0.1
3,3,5000,0.1
4,4,5000,0.1
5,5,5000,0.1
6,6,5000,0.1
7,7,5000,0.1
8,8,5000,0.1
9,9,5000,0.1



🔹 Muestra de imágenes
----------------------------------------
ℹ️                               Dimensión  \
0  [32x32, 32x32, 32x32, 32x32, 32x32]   
1  [32x32, 32x32, 32x32, 32x32, 32x32]   
2  [32x32, 32x32, 32x32, 32x32, 32x32]   

                                   Etiqueta  
0    [frog, truck, truck, deer, automobile]  
1  [automobile, ship, dog, automobile, dog]  
2         [cat, ship, ship, airplane, frog]  


## Preprocesamiento y `tf.data` (ONE-HOT)

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

if USE_EFFICIENTNET:
    from tensorflow.keras.applications.efficientnet import EfficientNetB0 as BASE, preprocess_input
    base_name = 'EfficientNetB0'
else:
    from tensorflow.keras.applications.resnet50 import ResNet50 as BASE, preprocess_input
    base_name = 'ResNet50'

'''
Aumentamos levemente el conjunto de datos de
entrenamiento para prevenir el sobreajuste'''

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.05),
], name="data_augmentation")

def preprocess_images(x):
    x = tf.cast(x, tf.float32)
    x = tf.image.resize(x, IMG_SIZE)
    x = preprocess_input(x)
    x = tf.reshape(x, (IMG_SIZE[0], IMG_SIZE[1], 3))
    return x

def make_dataset(x, y, training=True):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if training:
        ds = ds.shuffle(10000, seed=SEED, reshuffle_each_iteration=True)

    # Aplicamos preprocesamiento a las imágenes
    ds = ds.map(lambda img, label: (preprocess_images(img), label), num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        # Aplicamos un aumento de datos solo a las imágenes
        ds = ds.map(lambda img, label: (data_augmentation(img), label), num_parallel_calls=tf.data.AUTOTUNE)

    # Convertimos etiquetas ONE-HOT
    ds = ds.map(lambda img, label: (img, tf.one_hot(label, depth=NUM_CLASSES, dtype=tf.float32)), num_parallel_calls=tf.data.AUTOTUNE)


    ds = ds.batch(BATCH_SIZE, drop_remainder=True) #batches cerrados
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(x_tr, y_tr, training=True)
val_ds   = make_dataset(x_val, y_val, training=False)
test_ds  = make_dataset(x_test, y_test, training=False)

#Verificacion rápida de rangos y tipos
xb, yb = next(iter(train_ds.take(1)))
x_val, y_val = next(iter(val_ds.take(1)))
x_test, y_test = next(iter(test_ds.take(1)))

print_header("Rango y tipo de datos")
df_norm = pd.DataFrame({
    "Conjunto": ["Entrenamiento", "Validación", "Prueba"],
    "Shape": [xb.shape, x_val.shape, x_test.shape],
    "Rango_min": [tf.reduce_min(xb).numpy(), tf.reduce_min(x_val).numpy(), tf.reduce_min(x_test).numpy()],
    "Rango_max": [tf.reduce_max(xb).numpy(), tf.reduce_max(x_val).numpy(), tf.reduce_max(x_test).numpy()],
    "Tipo": [xb.dtype, x_val.dtype, x_test.dtype]})
display(df_norm)

print("============================================")
print_subheader("Visualización de un batch de imágenes")
print("xb:", xb.shape, xb.dtype, tf.reduce_min(xb).numpy(), tf.reduce_max(xb).numpy())
print("yb:", yb.shape, yb.dtype, tf.reduce_min(yb).numpy(), tf.reduce_max(yb).numpy())



📊 RANGO Y TIPO DE DATOS


,Conjunto,Shape,Rango_min,Rango_max,Tipo
0,Entrenamiento,"(64, 224, 224, 3)",0.0,255.0,<dtype: 'float32'>
1,Validación,"(64, 224, 224, 3)",0.0,255.0,<dtype: 'float32'>
2,Prueba,"(64, 224, 224, 3)",0.0,255.0,<dtype: 'float32'>



🔹 Visualización de un batch de imágenes
----------------------------------------
xb: (64, 224, 224, 3) <dtype: 'float32'> 0.0 255.0
yb: (64, 10) <dtype: 'float32'> 0.0 1.0


## Definición del modelo

In [7]:
IMG_SIZE = 224

if 'base_model' not in locals():
    base_model = EfficientNetB0(weights='imagenet',
                                include_top=False,
                                input_shape=(IMG_SIZE, IMG_SIZE, 3))


base_model.trainable = False  # Fase 1: congelado

print_header("Modelo base")
inputs = tf.keras.Input(shape= (IMG_SIZE, IMG_SIZE,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs, outputs, name=f'{base_name}_CIFAR10')

try:
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="categorical_crossentropy", # ONE-HOT
                  metrics=["accuracy", tf.keras.metrics.Precision(name="precision")], # Fix: Corrected "Acurracy" to "accuracy"
                  jit_compile=True)
except TypeError:
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="categorical_crossentropy", # ONE-HOT
                  metrics=["accuracy", tf.keras.metrics.Precision(name="precision")]) # Fix: Corrected "Acurracy" to "accuracy"
    print_info("jit_compile no disponible para esta version de TF")

model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

📊 MODELO BASE


Model: "EfficientNetB0_CIFAR10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

#División y codificación de etiquetas

In [8]:
#Dividir datos ( con variables x_tr, y_tr, x_val, y_val)

df_split= pd.DataFrame ({
    "conjunto":["Entrenamiento", "Validación", "Prueba"],
    "x_shape":[x_tr.shape, x_val.shape, x_test.shape],
    "y_shape (ONE-HOT)":[y_tr.shape, y_val.shape, y_test.shape],
    "cantidad":[len(x_tr), len(x_val), len(x_test)]
})

print_subheader("División de datos")
display(df_split)


#Dataset para entrenar

IMG_SIZE = 224
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

def pre_map_one_hot(x, y):
  x= tf.image.resize(x, (IMG_SIZE, IMG_SIZE))
  x= preprocess_input(x)
  x = tf.cast(x, tf.float32)
  y = tf.cast(y, tf.int32)
  y = tf.one_hot(y, depth=NUM_CLASSES, dtype=tf.float32)
  return x, y


train_ds = (tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
            .shuffle(10000, seed=RANDOM_STATE)
            .map(pre_map_one_hot, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTOTUNE))
val_ds = (tf.data.Dataset.from_tensor_slices((x_val, y_val))
          .map(pre_map_one_hot, num_parallel_calls=AUTOTUNE)
          .batch(BATCH_SIZE, drop_remainder=True)
          .prefetch(AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
           .map(pre_map_one_hot, num_parallel_calls=AUTOTUNE)
           .batch(BATCH_SIZE, drop_remainder=True)
           .prefetch(AUTOTUNE))

xb, yb = next(iter(train_ds.take(1)))
xb, yb = next(iter(val_ds.take(1)))
xb, yb = next(iter(test_ds.take(1)))


df_pipe = pd.DataFrame({
    "Conjunto": ["Entrenamiento", "Validación", "Prueba"],
    "x_shape": [tuple(xb.shape), tuple(xb.shape), tuple(xb.shape)],
    "y_shape": [tuple(yb.shape), tuple(yb.shape), tuple(yb.shape)],
    "Rango_minX": [float(tf.reduce_min(xb)), float(tf.reduce_min(xb)), float(tf.reduce_min(xb))],
    "Rango_maxX": [float(tf.reduce_max(xb)), float(tf.reduce_max(xb)), float(tf.reduce_max(xb))],
    "Tipo": [xb.dtype, xb.dtype, xb.dtype]
})

print_subheader("Pipeline de preprocesamiento")
display(df_pipe)


🔹 División de datos
----------------------------------------


,conjunto,x_shape,y_shape (ONE-HOT),cantidad
0,Entrenamiento,"(40000, 32, 32, 3)","(40000,)",40000
1,Validación,"(64, 224, 224, 3)","(64, 10)",64
2,Prueba,"(64, 224, 224, 3)","(64, 10)",64



🔹 Pipeline de preprocesamiento
----------------------------------------


,Conjunto,x_shape,y_shape,Rango_minX,Rango_maxX,Tipo
0,Entrenamiento,"(64, 224, 224, 3)","(64, 10, 10)",0.0,255.0,<dtype: 'float32'>
1,Validación,"(64, 224, 224, 3)","(64, 10, 10)",0.0,255.0,<dtype: 'float32'>
2,Prueba,"(64, 224, 224, 3)","(64, 10, 10)",0.0,255.0,<dtype: 'float32'>


## Entrenamiento — Fase 1 (base congelada)

In [ ]:
# ============================
# Callbacks y Entrenamiento (Fase 1 - congelada)
# ============================

# Evitamos conflictos con XLA/layout
tf.config.optimizer.set_jit(False)
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})

cbs = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy'),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2, verbose=1, monitor='val_loss'),
]

# Intento de forzar 1 GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')  # solo la primera GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Usando GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FREEZE,
    callbacks=cbs,
    verbose=2
)


Epoch 1/5


## Fine-Tuning — Fase 2 (descongelar cola final)

In [ ]:
# Descongelar el ~20% final de la base (excepto BatchNorm)
trainable_layers = int(len(base_model.layers) * 0.2)
for layer in base_model.layers[-trainable_layers:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history2 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FINETUNE, verbose=2)
train_time = time.time() - t0

## Evaluación en test

In [ ]:

# ============================
# Evaluación y reportes
# ============================
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test -> loss: {test_loss:.4f} | accuracy: {test_acc:.4f}")
test_metrics = model.evaluate(test_ds, verbose=0)
print(f"Test -> loss: {test_metrics[0]:.4f} | accuracy: {test_metrics[1]:.4f} | precision: {test_metrics[2]:.4f}")

# Predicciones y reportes
y_true = []
y_pred = []
for _, (xb, yb) in enumerate(test_ds):
    yp = model.predict(xb, verbose=0)
    y_pred.extend(np.argmax(yp, axis=1))
    y_true.extend(np.argmax(yb.numpy(), axis=1))

print("\\n=== Classification Report ===")
print(classification_report(y_true, y_pred, digits=4))

print("=== Matriz de Confusión ===")
print(confusion_matrix(y_true, y_pred))


## Curvas de entrenamiento (accuracy y loss)

In [ ]:
def plot_history(histories, metric='accuracy'):
    plt.figure(figsize=(6,4))
    for h in histories:
        plt.plot(h.history[metric])
    for h in histories:
        plt.plot(h.history['val_'+metric], linestyle='--')
    plt.title(f'Métrica: {metric}')
    plt.xlabel('Épocas')
    plt.ylabel(metric)
    plt.legend([f'train (f{i+1})' for i in range(len(histories))] +
               [f'val (f{i+1})' for i in range(len(histories))], loc='best')
    plt.show()

plot_history([history1, history2], metric='accuracy')
plot_history([history1, history2], metric='loss')

## Predicciones vs Reales (grid)

In [ ]:
test_batch = next(iter(test_ds))
images, labels = test_batch
pred_probs = model.predict(images, verbose=0)
preds = np.argmax(pred_probs, axis=1)

def show_grid(images, y_true, y_pred, n=12):
    n = min(n, images.shape[0])
    plt.figure(figsize=(12,6))
    for i in range(n):
        plt.subplot(3, n//3, i+1)
        # Se muestran imagen en rango [0,1] de forma segura
        plt.imshow(tf.cast(tf.clip_by_value(images[i]/255.0, 0, 1), tf.float32))
        title = f"T:{class_names[y_true[i]]}\nP:{class_names[y_pred[i]]}"
        plt.title(title, fontsize=9, color=('green' if y_true[i]==y_pred[i] else 'red'))
        plt.axis('off')
    plt.tight_layout(); plt.show()

show_grid(images, labels.numpy(), preds, n=12)

## Matriz de Confusión y Reporte de Clasificación

In [ ]:
# Predicciones completas sobre test
y_true_all, y_pred_all = [], []
for ims, labs in test_ds:
    pp = model.predict(ims, verbose=0)
    y_true_all.append(labs.numpy())
    y_pred_all.append(np.argmax(pp, axis=1))
y_true_all = np.concatenate(y_true_all)
y_pred_all = np.concatenate(y_pred_all)

cm = confusion_matrix(y_true_all, y_pred_all)

def plot_confusion_matrix(cm, classes, normalize=False, title='Matriz de Confusión'):
    if normalize:
        cm = cm.astype('float') / (cm.sum(axis=1, keepdims=True) + 1e-12)
    plt.figure(figsize=(7,6))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha='right')
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() * 0.6
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment='center',
                 color=('white' if cm[i, j] > thresh else 'black'))
    plt.ylabel('Real'); plt.xlabel('Predicho')
    plt.tight_layout(); plt.show()

plot_confusion_matrix(cm, class_names, normalize=False, title='Matriz de Confusión (Absoluta)')
plot_confusion_matrix(cm, class_names, normalize=True,  title='Matriz de Confusión (Normalizada)')

print('\n=== Reporte de Clasificación (Test) ===')
print(classification_report(y_true_all, y_pred_all, target_names=class_names))

## Resumen final (para informe)

In [ ]:
print('\n' + '='*60)
print(f"Modelo base: {base_name}")
print(f"Fase freeze epochs: {EPOCHS_FREEZE} | Fine-tune epochs: {EPOCHS_FINETUNE}")
print(f"Accuracy test: {test_acc:.4f} | Loss test: {test_loss:.4f}")
print(f"Tiempo total de entrenamiento: {train_time/60:.1f} min")
print('='*60)

## Análisis
**Arquitectura elegida:** EfficientNetB0 (conmutables a ResNet50 cambiando `USE_EFFICIENTNET=False`).  
**Justificación:** EfficientNetB0 ofrece una gran relación precisión/eficiencia gracias al *compound scaling*. ResNet50 es un estándar robusto con *skip connections* que facilitan el entrenamiento profundo. Para el alcance de esta actividad y recursos moderados, EfficientNetB0 suele converger más rápido y con menor costo computacional.

**Principales desafíos:**
1. Ajustar el tamaño de entrada y usar el `preprocess_input` correcto.
2. Evitar *overfitting* durante el fine-tuning (LR bajo, capas limitadas, Dropout).
3. Balancear tiempo de entrenamiento vs rendimiento (pipelines `tf.data`).

**Mejoras para producción (ideas):**
- Exportar a **TensorFlow Lite** y cuantizar (float16/int8) para móvil/edge.
- Integrar **EarlyStopping**/**ReduceLROnPlateau** y **data augmentation** más amplio.
- Explicabilidad con **Grad-CAM** si el dominio lo requiere (p. ej., médico/industrial).
- Monitoreo en despliegue: *drift*, latencia, *A/B testing*.


## README
### Estructura
- `notebooks/Plantilla.ipynb`: flujo completo de Transfer Learning (este archivo).
- `data/` (opcional): datasets locales si no usas CIFAR-10.
- `models/` (opcional): pesos exportados / TFLite.

### Librerías y para qué se usaron
- **tensorflow / keras**: carga del dataset, definición del modelo base (ResNet50/EfficientNetB0), *transfer learning*, entrenamiento y evaluación.
- **numpy**: manejo de arreglos y utilidades numéricas.
- **matplotlib.pyplot**: visualización de curvas, grid de predicciones y matrices.
- **sklearn.metrics**: `confusion_matrix`, `classification_report` para evaluación detallada.

### Resultados esperados (CIFAR-10)
- Convergencia rápida en pocas épocas con la base congelada.
- Mejora moderada tras el fine-tuning.
- Visualizaciones generadas: **accuracy**, **loss**, **grid predicciones**, **matriz de confusión** y **reporte de clasificación**.